# Beijing Air-Quality Time Series Project
### Modeling Time Series

by Dolci Sanders and Paul Torres



In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
from pylab import savefig 

import glob
from sklearn.model_selection import train_test_split
import pickle

from statsmodels.tsa.arima_model import ARIMA
import pmdarima

# Styling Guide 
plt.style.use('fivethirtyeight')
sns.set_palette(sns.diverging_palette(250,700, s= 70 , l= 10, n= 12))


## Read in pickle, check index for date time

In [3]:
train = pd.read_pickle('PKL/train.pkl')
test = pd.read_pickle('PKL/test.pkl')
time = pd.read_pickle('PKL/time.pkl')

In [4]:
train.index

DatetimeIndex(['2013-03-01 00:00:00', '2013-03-01 01:00:00',
               '2013-03-01 02:00:00', '2013-03-01 03:00:00',
               '2013-03-01 04:00:00', '2013-03-01 05:00:00',
               '2013-03-01 06:00:00', '2013-03-01 07:00:00',
               '2013-03-01 08:00:00', '2013-03-01 09:00:00',
               ...
               '2016-03-26 04:00:00', '2016-03-26 05:00:00',
               '2016-03-26 06:00:00', '2016-03-26 07:00:00',
               '2016-03-26 08:00:00', '2016-03-26 09:00:00',
               '2016-03-26 10:00:00', '2016-03-26 11:00:00',
               '2016-03-26 12:00:00', '2016-03-26 13:00:00'],
              dtype='datetime64[ns]', name='Date', length=323016, freq=None)

In [5]:
train.index = pd.DatetimeIndex(train.index.values,
                               freq=train.index.inferred_freq)

test.index = pd.DatetimeIndex(test.index.values,
                               freq=test.index.inferred_freq)

In [6]:
train = train.drop(columns=['station'], axis = 1)
test = test.drop(columns=['station'], axis = 1)

In [7]:
train.index

DatetimeIndex(['2013-03-01 00:00:00', '2013-03-01 01:00:00',
               '2013-03-01 02:00:00', '2013-03-01 03:00:00',
               '2013-03-01 04:00:00', '2013-03-01 05:00:00',
               '2013-03-01 06:00:00', '2013-03-01 07:00:00',
               '2013-03-01 08:00:00', '2013-03-01 09:00:00',
               ...
               '2016-03-26 04:00:00', '2016-03-26 05:00:00',
               '2016-03-26 06:00:00', '2016-03-26 07:00:00',
               '2016-03-26 08:00:00', '2016-03-26 09:00:00',
               '2016-03-26 10:00:00', '2016-03-26 11:00:00',
               '2016-03-26 12:00:00', '2016-03-26 13:00:00'],
              dtype='datetime64[ns]', length=323016, freq=None)

In [ ]:
arima = ARIMA(train, order = (7,2,1))
arima_fit = arima.fit(disp=0)
print(arima.summary())

/Users/dolcikey/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/dolcikey/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)
/Users/dolcikey/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/dolcikey/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will b